In [1]:
!pip install psycopg2

In [2]:
import csv
import psycopg2
import pandas as pd

In [3]:
%load_ext sql

In [6]:
%sql sqlite:///my_database.db

In [9]:
import sqlite3

conn = sqlite3.connect("test.db")
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER, name TEXT)")
cursor.execute("INSERT INTO users VALUES (1, 'Alice')")
conn.commit()

cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

[(1, 'Alice'), (1, 'Alice')]


In [16]:
# function to read from database
def read(conn, read_):
    print('Read')
    cursor = conn.cursor()
    cursor.execute(read_)
    for row in cursor:
        print(f'row = {row}')
    print()

# function to create in postgre database
def create(conn, create_):
    cursor = conn.cursor() # create cursor object
    cursor.execute(create_) # execute query
    conn.commit() # commit query to database
    print('Table have been created successfull!!!')
    #read(conn)

# function to insert in postgre database
def insert(conn, insert_):
    cursor = conn.cursor()
    cursor.execute(insert_)
    conn.commit()
    print('Records have been successfully inserted!!!')
    #read(conn)

# function to update table
def update(conn, update_):
    print('Update')
    cursor = conn.cursor()
    cursor.execute(update_)
    conn.commit()
    #read(conn)

# function to delete in postgre database
def delete(conn, delete_):
    print('Delete')
    cursor = conn.cursor()
    cursor.execute(delete_)
    conn.commit()
    #read(conn)

# close the cursor and connection to the server
def close():
    cursor.close()
    conn.close()

# function to create pandas dataframe
def create_pandas_df(sql_query, database=conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [20]:
def create(conn):
    cursor = conn.cursor()

    # First execute the DROP statement
    drop_query = "DROP TABLE IF EXISTS SpaceX;"
    cursor.execute(drop_query)

    # Then execute the CREATE TABLE statement
    create_query = '''
    CREATE TABLE SpaceX (
        Date DATE NULL,
        Time TIME NULL,
        BoosterVersion VARCHAR(50) NULL,
        LaunchSite VARCHAR(50) NULL,
        Payload VARCHAR(100) NULL,
        PayloadMassKG INT NULL,
        Orbit VARCHAR(50) NULL,
        Customer VARCHAR(100) NULL,
        MissionOutcome VARCHAR(50) NULL,
        LandingOutcome VARCHAR(100) NULL
    );
    '''
    cursor.execute(create_query)
    conn.commit()

In [27]:
import sqlite3

conn = sqlite3.connect("spacex.db")

create_table = '''
CREATE TABLE IF NOT EXISTS SpaceX (
    Date TEXT,
    Time TEXT,
    BoosterVersion TEXT,
    LaunchSite TEXT,
    Payload TEXT,
    PayloadMassKG INTEGER,
    Orbit TEXT,
    Customer TEXT,
    MissionOutcome TEXT,
    LandingOutcome TEXT
);
'''

cursor = conn.cursor()
cursor.execute(create_table)
conn.commit()

In [28]:
import csv

with open('Spacex.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        cursor.execute(
            "INSERT INTO SpaceX VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", row
        )
conn.commit()

In [31]:
read_ = '''
    SELECT *
    FROM   SpaceX
    LIMIT 10
    '''
def read(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
read(conn, read_)

('2010-04-06', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-08-12', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-08-10', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-01-03', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-09-23', '16:00:00', 'F9 v1.1  B1003', 'VAFB SLC-4E', 'CASSIOPE', 500, 'Polar LEO', 'MDA', 'Success', 'Uncontrolled (ocean)')
('2013-03-12', '22:41:00', 'F9 v1.1', 'CCAFS LC-40', 'SES-8', 3170, 'GTO', 'SES', 'Success', 'No attempt')
('2014

In [32]:
task_1 = '''
        SELECT DISTINCT LaunchSite
        FROM SpaceX
'''
create_pandas_df(task_1, database=conn)

,LaunchSite
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


In [33]:
task_2 = '''
        SELECT *
        FROM SpaceX
        WHERE LaunchSite LIKE 'CCA%'
        LIMIT 5
        '''
create_pandas_df(task_2, database=conn)

,Date,Time,BoosterVersion,LaunchSite,Payload,PayloadMassKG,Orbit,Customer,MissionOutcome,LandingOutcome
0,2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-08-10,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [34]:
task_3 = '''
        SELECT SUM(PayloadMassKG) AS Total_PayloadMass
        FROM SpaceX
        WHERE Customer LIKE 'NASA (CRS)'
        '''
create_pandas_df(task_3, database=conn)

,Total_PayloadMass
0,45596


In [35]:
task_4 = '''
        SELECT AVG(PayloadMassKG) AS Avg_PayloadMass
        FROM SpaceX
        WHERE BoosterVersion = 'F9 v1.1'
        '''
create_pandas_df(task_4, database=conn)

,Avg_PayloadMass
0,2928.4


In [36]:
task_5 = '''
        SELECT MIN(Date) AS FirstSuccessfull_landing_date
        FROM SpaceX
        WHERE LandingOutcome LIKE 'Success (ground pad)'
        '''
create_pandas_df(task_5, database=conn)

,FirstSuccessfull_landing_date
0,2015-12-22


In [37]:
task_6 = '''
        SELECT BoosterVersion
        FROM SpaceX
        WHERE LandingOutcome = 'Success (drone ship)'
            AND PayloadMassKG > 4000
            AND PayloadMassKG < 6000
        '''
create_pandas_df(task_6, database=conn)

,BoosterVersion
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


In [38]:
task_7a = '''
        SELECT COUNT(MissionOutcome) AS SuccessOutcome
        FROM SpaceX
        WHERE MissionOutcome LIKE 'Success%'
        '''

task_7b = '''
        SELECT COUNT(MissionOutcome) AS FailureOutcome
        FROM SpaceX
        WHERE MissionOutcome LIKE 'Failure%'
        '''
print('The total number of successful mission outcome is:')
display(create_pandas_df(task_7a, database=conn))
print()
print('The total number of failed mission outcome is:')
create_pandas_df(task_7b, database=conn)

The total number of successful mission outcome is:


,SuccessOutcome
0,100



The total number of failed mission outcome is:


,FailureOutcome
0,1


In [39]:
task_8 = '''
        SELECT BoosterVersion, PayloadMassKG
        FROM SpaceX
        WHERE PayloadMassKG = (
                                SELECT MAX(PayloadMassKG)
                                FROM SpaceX
                                )
        ORDER BY BoosterVersion
        '''
create_pandas_df(task_8, database=conn)

,BoosterVersion,PayloadMassKG
0,F9 B5 B1048.4,15600
1,F9 B5 B1048.5,15600
2,F9 B5 B1049.4,15600
3,F9 B5 B1049.5,15600
4,F9 B5 B1049.7,15600
5,F9 B5 B1051.3,15600
6,F9 B5 B1051.4,15600
7,F9 B5 B1051.6,15600
8,F9 B5 B1056.4,15600
9,F9 B5 B1058.3,15600


In [40]:
task_9 = '''
        SELECT BoosterVersion, LaunchSite, LandingOutcome
        FROM SpaceX
        WHERE LandingOutcome LIKE 'Failure (drone ship)'
            AND Date BETWEEN '2015-01-01' AND '2015-12-31'
        '''
create_pandas_df(task_9, database=conn)

,BoosterVersion,LaunchSite,LandingOutcome
0,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
1,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


In [41]:
task_10 = '''
        SELECT LandingOutcome, COUNT(LandingOutcome)
        FROM SpaceX
        WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'
        GROUP BY LandingOutcome
        ORDER BY COUNT(LandingOutcome) DESC
        '''
create_pandas_df(task_10, database=conn)

,LandingOutcome,COUNT(LandingOutcome)
0,No attempt,10
1,Success (drone ship),6
2,Success (ground pad),5
3,Failure (drone ship),5
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Precluded (drone ship),1
7,Failure (parachute),1
